# Оптимальные питоновские реализация оптимизации ARTM

# Оптимизация произвольной функции

# Thetaless оптимизация


In [1]:
import numpy as np
from numpy.core.umath_tests import inner1d
import scipy
import scipy.sparse
from sklearn.datasets import fetch_20newsgroups
import gensim
from collections import Counter
import heapq
import nltk
import random
from nltk.corpus import stopwords
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import time
%matplotlib inline

# Разные функции потерь

In [2]:
class LogFunction(object):
    def calc(self, x):
        return np.log(x + 1e-20)
    def calc_der(self, x):
        return 1. / (x + 1e-20)
    

class IdFunction(object):
    def calc(self, x):
        return x + 1e-20
    def calc_der(self, x):
        return np.ones_like(x)
    

class SquareFunction(object):
    def calc(self, x):
        return (x + 1e-20) ** 2
    def calc_der(self, x):
        return 2. * (x + 1e-20) ** 2
    

class CubeLogFunction(object):
    def calc(self, x):
        return np.log(x + 1e-20) ** 3
    def calc_der(self, x):
        return 3. * np.log(x + 1e-20) ** 2 / (x + 1e-20)
    

class SquareLogFunction(object):
    def calc(self, x):
        return np.log(x + 1e-20) * np.abs(np.log(x + 1e-20))
    def calc_der(self, x):
        return 2. * np.abs(np.log(x + 1e-20)) / (x + 1e-20)

    
class FiveLogFunction(object):
    def calc(self, x):
        return np.log(x + 1e-20) ** 5
    def calc_der(self, x):
        return 5. * np.log(x + 1e-20) ** 4 / (x + 1e-20)
    

class CubeRootLogFunction(object):
    def calc(self, x):
        return np.cbrt(np.log(x + 1e-20))
    def calc_der(self, x):
        return 1. / 3 / (np.cbrt(np.log(x + 1e-20)) ** 2) / (x + 1e-20)
    
    
class SquareRootLogFunction(object):
    def calc(self, x):
        return np.sqrt(- np.log(x + 1e-20))
    def calc_der(self, x):
        return 1. / 2. / np.sqrt(- np.log(x + 1e-20)) / (x + 1e-20)
    

class ExpFunction(object):
    def calc(self, x):
        return np.exp(x)
    def calc_der(self, x):
        return np.exp(x)

    
class EntropyFunction(object):
    def calc(self, x):
        return (np.log(x + 1e-20) + 50.) * (x + 1e-20)
    def calc_der(self, x):
        return np.log(x + 1e-20) + 50.

# Разные регуляризации

In [3]:
def trivial_regularization(n_tw, n_dt):
    return np.zeros_like(n_tw), np.zeros_like(n_dt)

def create_reg_decorr(tau, theta_alpha=0.):
    def fun(n_tw, n_dt):
        phi_matrix = n_tw / np.sum(n_tw, axis=1)[:, np.newaxis]
        theta_matrix = n_dt / np.sum(n_dt, axis=1)[:, np.newaxis]
        aggr_phi = np.sum(phi_matrix, axis=1)
        return - tau * np.transpose(phi_matrix * (aggr_phi[:, np.newaxis] - phi_matrix)), theta_alpha
    return fun

def create_reg_lda(phi_alpha, theta_alpha):
    def fun (n_tw, n_dt):
        return np.zeros_like(n_tw) + phi_alpha, np.zeros_like(n_dt) + theta_alpha
    return fun


# Подготовка Датасета

Нужно скачать некоторые коллекции данных и установить библиотеки (nltk, gensim)

In [4]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/tylorn/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
english_stopwords = set(stopwords.words('english'))

In [7]:
def prepare_dataset(dataset, calc_cooccurences=False, train_test_split=None, token_2_num=None):
    is_token_2_num_provided = token_2_num is not None 
    # remove stopwords
    if not is_token_2_num_provided:
        token_2_num = {}
        occurences = Counter()
        for i, doc in enumerate(dataset.data):
            tokens = gensim.utils.lemmatize(doc)
            for token in set(tokens):
                occurences[token] += 1
            if i % 500 == 0:
                print 'Processed: ', i, 'documents from', len(dataset.data)
    
    row, col, data = [], [], []
    row_test, col_test, data_test = [], [], []
    not_empty_docs_number = 0
    doc_targets = []
    doc_cooccurences = Counter()
    doc_occurences = Counter()
    random_gen = random.Random(42)
    
    for doc, target in zip(dataset.data, dataset.target):
        tokens = gensim.utils.lemmatize(doc)
        cnt = Counter()
        cnt_test = Counter()
        for token in tokens:
            word = token.split('/')[0]
            if not is_token_2_num_provided and word not in english_stopwords and 3 <= occurences[token] and token not in token_2_num:
                token_2_num[token] = len(token_2_num)
            if token in token_2_num:
                if train_test_split is None or random_gen.random() < train_test_split:
                    cnt[token_2_num[token]] += 1
                else:
                    cnt_test[token_2_num[token]] += 1
        
        if len(cnt) > 0 and (train_test_split is None or len(cnt_test) > 0):
            for w, c in cnt.iteritems():
                row.append(not_empty_docs_number)
                col.append(w)
                data.append(c)
                
            for w, c in cnt_test.iteritems():
                row_test.append(not_empty_docs_number)
                col_test.append(w)
                data_test.append(c)
                
            not_empty_docs_number += 1
            doc_targets.append(target)
            
            if calc_cooccurences:
                words = set(cnt.keys() + cnt_test.keys())
                doc_occurences.update(words)
                doc_cooccurences.update({(w1, w2) for w1 in words for w2 in words if w1 != w2})
        
    num_2_token = {
        v: k
        for k, v in token_2_num.iteritems()
    }
    print 'Nonzero values:', len(data)
    shape = (len(doc_targets), len(token_2_num))
    
    if train_test_split is None:
        if calc_cooccurences:
            return scipy.sparse.csr_matrix((data, (row, col)), shape=shape), token_2_num, num_2_token, doc_targets, doc_occurences, doc_cooccurences
        else:
            return scipy.sparse.csr_matrix((data, (row, col)), shape=shape), token_2_num, num_2_token, doc_targets
    else:
        if calc_cooccurences:
            return (
                scipy.sparse.csr_matrix((data, (row, col)), shape=shape),
                scipy.sparse.csr_matrix((data_test, (row_test, col_test)), shape=shape),
                token_2_num,
                num_2_token,
                doc_targets,
                doc_occurences,
                doc_cooccurences
            )
        else:
            return (
                scipy.sparse.csr_matrix((data, (row, col)), shape=shape),
                scipy.sparse.csr_matrix((data_test, (row_test, col_test)), shape=shape),
                token_2_num,
                num_2_token,
                doc_targets
            )


# Вычисление правдоподобных функций

### имеется в виду вычисление функций вида $\sum_{dw} n_{dw} f(\sum_{t} \phi_{wt} \theta_{td})$

##### Ключевой момент - использование функции inner1d. Она позволяет перемножить попарно строчки матриц, не сохраняя промежуточное состояние. А индексация в numpy не создаёт новый массив, делает view над ним. Таким образом, подсчёт $\sum_{t} \phi_{wt} \theta_{td}$  делается максимально эффективным способом и по времени и по памяти.

In [19]:
def create_calculate_likelihood_like_function(n_dw_matrix, loss_function=LogFunction()):
    D, W = n_dw_matrix.shape
    docptr = []
    indptr = n_dw_matrix.indptr
    for doc_num in xrange(D):
        docptr.extend([doc_num] * (indptr[doc_num + 1] - indptr[doc_num]))
    docptr = np.array(docptr)
    wordptr = n_dw_matrix.indices
    
    def fun(phi_matrix, theta_matrix):
        s_data = loss_function.calc(inner1d(theta_matrix[docptr, :], np.transpose(phi_matrix)[wordptr, :]))
        return np.sum(n_dw_matrix.data * s_data)

    return fun

# EM алгоритм

## Общая схема:
#### Неоходимо сначала вычислить $p_{tdw} = \frac{\phi_{wt} \theta_{td}}{\sum_s \phi_{ws} \theta_{sd}}$
#### Считаем $n_{wt} = \sum_d n_{dw} p_{tdw}$ и $n_{td} = \sum_w n_{dw} p_{tdw}$
#### Вычисляем $r_{wt}, r_{td}$ как функцию от $n_{wt}, n_{td}$
#### Прибавляем, делаем положительную срезку и нормируем

## Оптимизация вычисления:
#### Обозначим за $s_{dw}$ следующее выражение $\sum_t \phi_{wt} \theta_{td}$, фактически это наше предсказание для вероятности
####  Тогда $p_{tdw} = \frac{\phi_{wt} \theta_{td}}{s_{dw}}$
#### Подставим это выражение например в $n_wt$
#### И получим, что $n_{wt} = \sum_d n_{dw} \frac{\phi_{wt} \theta_{td}}{s_{dw}} = \phi_{wt} \sum_d \theta_{td} \cdot \frac{n_{dw}}{s_{dw}}$, аналогично $n_{td} = \theta_{td} \sum_w \phi_{wt} \cdot \frac{n_{dw}}{s_{dw}}$
#### Таким образом, мы видим, что фактически нам нужно знать матрицу $\frac{n_{dw}}{s_{dw}}$, а она очень разреженная, поэтому и $s_{dw}$ нужно не для всех пар вычислять, а только там, где $n_{dw} > 0$. 
#### То есть нам нужно эффективно закодить вычисление разженной матрицы $s_{dw}$ (матрица $n_{dw}$ уже есть в разреженном виде, так как подаётся на вход алгоритма), а затем просто поэлементно поделить
#### Причём хочется, чтобы промежуточные значения $p_{tdw}$ не сохранялись (как мы увидели, они в конечном варианте не важны)
#### Обозначим эту матрицу за $A$. Тогда $n_{wt} = \phi_{wt} (\Theta A)_{tw}$, а $n_{td} = \theta_{td} (A \Phi^T)_{dt}$.
#### Перемножить разреженную матрицу на плотную можно быстро, если правильно её хранить (по строкам, или по столбцам)
#### Если оптимизируется не правдоподобие, какая-то другая функция вида $\sum_{dw} n_{dw} f(s_{dw})$ (правдоподобие будет, если $f(x) = \ln x$ ) , то в этом случае нужно определить матрицу $A$ как $A_{dw} = n_{dw} f'(s_{dw})$

In [20]:
def em_optimization(
    n_dw_matrix, 
    phi_matrix,
    theta_matrix,
    regularization_list,
    iters_count=100,
    loss_function=LogFunction(),
    iteration_callback=None,
    const_phi=False
):
    D, W = n_dw_matrix.shape
    T = phi_matrix.shape[0]
    phi_matrix = np.copy(phi_matrix)
    theta_matrix = np.copy(theta_matrix)
    docptr = []
    indptr = n_dw_matrix.indptr
    for doc_num in xrange(D):
        docptr.extend([doc_num] * (indptr[doc_num + 1] - indptr[doc_num]))
    docptr = np.array(docptr)
    wordptr = n_dw_matrix.indices
    
    start_time = time.time()
    for it in xrange(iters_count):
        phi_matrix_tr = np.transpose(phi_matrix)
        # следующая строчка это 60% времени работы алгоритма
        s_data = loss_function.calc_der(inner1d(theta_matrix[docptr, :], phi_matrix_tr[wordptr, :]))
        # следующая часть это 25% времени работы алгоритма
        A = scipy.sparse.csr_matrix(
            (
                n_dw_matrix.data * s_data, 
                n_dw_matrix.indices, 
                n_dw_matrix.indptr
            ), 
            shape=n_dw_matrix.shape
        )
        A_tr = A.tocsc().transpose()
        # Остальное это 15% времени
        n_tw = np.transpose(A_tr.dot(theta_matrix)) * phi_matrix
        n_dt = A.dot(phi_matrix_tr) * theta_matrix
        
        r_tw, r_dt = regularization_list[it](n_tw, n_dt)
        n_tw += r_tw
        n_dt += n_dt
        n_tw[n_tw < 0] = 0
        n_dt[n_dt < 0] = 0
        
        if not const_phi:
            phi_matrix = n_tw / np.sum(n_tw, axis=1)[:, np.newaxis]
        theta_matrix = n_dt / np.sum(n_dt, axis=1)[:, np.newaxis]
        
        if iteration_callback is not None:
            iteration_callback(it, phi_matrix, theta_matrix)
    
    print 'Iters time', time.time() - start_time
    return phi_matrix, theta_matrix

# Naive thetaless EM


### Основная идея: давайте вообще не хранить $\Theta$, а вместо этого вычислять её на лету одной итерацией ЕМ алгоритма, которую можно легко выписать.

##### Пусть тематический профиль документа инициализирован равномерно, то для этого документа $p_{tdw} = \frac{\phi_{wt}}{\sum_s \phi_s} \equiv \overline{\phi}_{wt} \equiv (\overline{\Phi})_{wt} \equiv p(t~|~w)$ . Эту матрицу легко рассчитать.
##### На первой итерации  будет подсчитано $n_{td} = \sum_{d} n_{dw} p_{tdw} = \sum_{d} n_{dw} (\overline{\Phi})_{wt} = (N\overline{\Phi})_{dt}$
##### И, соответственно, $\theta_{td} = \frac{n_{td}}{\sum_t n_{td}} =  \frac{n_{td}}{n_d}$
##### Введём матрицу $B_{dw} \equiv \frac{n_{dw}}{n_d}$, тогда $\Theta = B \overline{\Phi}$ 
##### Идеологически, мы зафиксировали, что $\Theta$ - детерминированная функция от $\Phi$. И теперь оптимизируем не $L(\Phi, \Theta)$, а $\overline{L}(\Phi) = L(\Phi, B \overline{\Phi})$
##### Наивность решения состоит в том, что мы полностью игнорируем любые действия с $\Theta$ на М шаге. То есть мы не считаем $n_{td}$ и не обновляем $\theta_{td}$ (этой матрицы вообще нет). А с $n_{wt}$ мы поступаем также как на обычном М шаге. Регуляризаторы на $\Phi$ обрабатываются точно также как и раньше ($r_{wt}$ прибавляется к $n_{wt}$), а регуляризаторы $\Theta$ игнорируются.

In [9]:
def naive_thetaless_em_optimization(
    n_dw_matrix, 
    phi_matrix,
    regularization_list,
    iters_count=100,
    iteration_callback=None
):
    D, W = n_dw_matrix.shape
    T = phi_matrix.shape[0]
    phi_matrix = np.copy(phi_matrix)
    docptr = []
    indptr = n_dw_matrix.indptr
    for doc_num in xrange(D):
        docptr.extend([doc_num] * (indptr[doc_num + 1] - indptr[doc_num]))
    docptr = np.array(docptr)
    wordptr = n_dw_matrix.indices
    
    start_time = time.time()
    for it in xrange(iters_count):
        phi_rev_matrix = np.transpose(phi_matrix / np.sum(phi_matrix, axis=0))
        theta_matrix = n_dw_matrix.dot(phi_rev_matrix)
        theta_matrix /= np.sum(theta_matrix, axis=1)[:, np.newaxis]
        phi_matrix_tr = np.transpose(phi_matrix)
        
        s_data = 1. / inner1d(theta_matrix[docptr, :], phi_matrix_tr[wordptr, :])
        A = scipy.sparse.csr_matrix(
            (
                n_dw_matrix.data  * s_data , 
                n_dw_matrix.indices, 
                n_dw_matrix.indptr
            ), 
            shape=n_dw_matrix.shape
        ).tocsc()
            
        n_tw = (A.T.dot(theta_matrix)).T * phi_matrix
        r_tw, _ = regularization_list[it](n_tw, theta_matrix)
        n_tw += r_tw
        n_tw[n_tw < 0] = 0
        phi_matrix = n_tw / np.sum(n_tw, axis=1)[:, np.newaxis]

        if iteration_callback is not None:
            iteration_callback(it, phi_matrix, theta_matrix)
    
    print 'Iters time', time.time() - start_time    
    return phi_matrix, theta_matrix

# ARTM thetaless EM optimization

##### Данное решение исправляет "наивность" предыдущего подхода, учитывая зависимость от $\Theta$. Фактически, это можно написать в виде регуляризатора. Чтобы понять, как именно это сделать, вспомним как работает EM алгоритм.

##### На каждой итерации сначала определяются $p_{tdw}$, фиксируются, а затем строится функционал нижней оценки: $Q(\Phi, \Theta) = \sum_{dtw} n_{dw} p_{tdw} \left( \ln \phi_{wt} + \ln \theta_{td}\right) + R(\Phi, \Theta)$. Цель М-шага увеличить значение данного функционала по сравнению с $\Phi$ и $\Theta$ с предыдущей итерации.

##### Несмотря на то, что теперь $\Theta$ это функция от $\Phi$, тот факт, что это всё ещё нижняя оценка, никуда не пропадает. Поэтому теперь наша цель подобрать $\Phi$, чтобы увеличить значение по сравнению с $\Phi$ с предыдущей итерации следующий функционал: 

$\sum_{dtw} n_{dw} p_{tdw} \left( \ln \phi_{wt} + \ln (\Theta(\Phi))_{dt}\right) + R(\Phi, \Theta(\Phi))$.

##### Возьмём производные как обычно

##### $\frac{\partial{Q}}{\partial{\phi_{vr}}} = \frac{1}{\phi_{vr}} \left( \sum_{d} n_{dv} p_{rdv} + \phi_{vr} \frac{\partial{R}}{\partial{\phi_{vr}}} + \sum_{dtw} n_{dw} p_{tdw} \frac{1}{\theta_{td}} \frac{\partial{\theta_{td}}}{\partial{\phi_{vr}}} +  \sum_{dt} \frac{\partial{R}}{\partial{\theta_{td}}} \frac{\partial{\theta_{td}}}{\partial{\phi_{vr}}} \right)$

##### Так как $p_{tdw} = \frac{\phi_{wt} \theta_{td}}{\sum_s \phi_{ws} \theta_{sd}}$, то третье слагаемое можно упростить

##### $\sum_{dtw} n_{dw} p_{tdw} \frac{1}{\theta_{td}} \frac{\partial{\theta_{td}}}{\partial{\phi_{vr}}} = \sum_{dtw} n_{dw}\frac{\phi_{wt}}{\sum_s \phi_{ws} \theta_{sd}} \frac{\partial{\theta_{td}}}{\partial{\phi_{vr}}} = \sum_{dtw} A_{dw} \phi_{wt} \frac{\partial{\theta_{td}}}{\partial{\phi_{vr}}}$

##### Итого

##### $\frac{\partial{Q}}{\partial{\phi_{vr}}} = \frac{1}{\phi_{vr}} \left( \sum_{d} n_{dv} p_{rdv} + \phi_{vr}\left( \frac{\partial{R}}{\partial{\phi_{vr}}} + \sum_{dtw} A_{dw} \phi_{wt} \frac{\partial{\theta_{td}}}{\partial{\phi_{vr}}} +  \sum_{dt} \frac{\partial{R}}{\partial{\theta_{td}}} \frac{\partial{\theta_{td}}}{\partial{\phi_{vr}}} \right) \right)$

##### Обозначим за $C = A \Phi^T + \frac{\partial{R}}{\partial{\Theta}}$, тогда
$\frac{\partial{Q}}{\partial{\phi_{vr}}} = \frac{1}{\phi_{vr}} \left( \sum_{d} n_{dv} p_{rdv} + \phi_{vr}\left( \frac{\partial{R}}{\partial{\phi_{vr}}} + \sum_{dt} C_{dt} \frac{\partial{\theta_{td}}}{\partial{\phi_{vr}}} \right) \right)$

##### Как видим, получившийся остаток фактически и есть требуемый регуяризатор на $\Phi$. Осталось только найти $\frac{\partial{\theta_{td}}}{\partial{\phi_{vr}}}$

##### $\theta_{td} = \sum_{w} B_{dw} \frac{\phi_{wt}}{\sum_s \phi_{ws}}$. Обозначим $\frac{1}{\sum_s \phi_{ws}}$ за $norm_w$, тогда

$\theta_{td} = \sum_{w} B_{dw} \phi_{wt} norm_w$

$\frac{\partial{\theta_{td}}}{\partial{\phi_{vr}}} =  \sum_{w} B_{dw}~norm_w \delta_{vwrt} +  \sum_{w} B_{dw} \phi_{wt} \frac{\partial{norm_w}}{\partial{\phi_{vr}}} = 
\sum_{w} B_{dw} norm_w \delta_{vwrt} - \sum_{w} B_{dw}~\phi_{wt}~norm_w^2~\delta_{vw} =
B_{dv}~norm_v~\delta_{rt} - B_{dv}~\phi_{vt}~norm_w^2
$

##### Тут $\delta$ это символ Кронекера

##### Теперь

$\sum_{dt} C_{dt} \frac{\partial{\theta_{td}}}{\partial{\phi_{vr}}} = \sum_{dt} C_{dt} \left( B_{dv}~norm_v~\delta_{rt} - B_{dv}~\phi_{vt}~norm_v^2 \right) =  norm_v~\sum_d C_{dr} B_{dv} -  norm_v^2~\sum_{dt} C_{dt} B_{dv} \phi_{vt} = norm_v (C^T B)_{rv} - norm_v^2 (\Phi^T C^T B)_{vv}$

##### В numpy можно вычислить только диагональ при помощи einsum, поэтому эту регуляризационную добавку можно эффективно вычислить


In [26]:
def artm_thetaless_em_optimization(
    n_dw_matrix, 
    phi_matrix,
    regularization_list,
    iters_count=100,
    iteration_callback=None
):
    D, W = n_dw_matrix.shape
    T = phi_matrix.shape[0]
    phi_matrix = np.copy(phi_matrix)
    docptr = []
    docsizes = []
    indptr = n_dw_matrix.indptr
    for doc_num in xrange(D):
        size = indptr[doc_num + 1] - indptr[doc_num]
        docptr.extend([doc_num] * size)
        #docsizes.extend([size] * size)
        docsizes.extend([np.sum(n_dw_matrix.data[indptr[doc_num]:indptr[doc_num + 1]])] * size)
    docptr = np.array(docptr)
    wordptr = n_dw_matrix.indices
    docsizes = np.array(docsizes)
    
    B = scipy.sparse.csr_matrix(
        (
            1. * n_dw_matrix.data  / docsizes, 
            n_dw_matrix.indices, 
            n_dw_matrix.indptr
        ), 
        shape=n_dw_matrix.shape
    ).tocsc()
    
    start_time = time.time()
    for it in xrange(iters_count):
        word_norm = np.sum(phi_matrix, axis=0)
        word_norm[word_norm == 0] = 1e-20
        phi_rev_matrix = np.transpose(phi_matrix / word_norm)
        
        theta_matrix = n_dw_matrix.dot(phi_rev_matrix)
        theta_matrix /= np.sum(theta_matrix, axis=1)[:, np.newaxis]
        phi_matrix_tr = np.transpose(phi_matrix)
        
        s_data = 1. / inner1d(theta_matrix[docptr, :], phi_matrix_tr[wordptr, :])
        A = scipy.sparse.csr_matrix(
            (
                n_dw_matrix.data  * s_data , 
                n_dw_matrix.indices, 
                n_dw_matrix.indptr
            ), 
            shape=n_dw_matrix.shape
        ).tocsc()
            
        n_tw = A.T.dot(theta_matrix).T * phi_matrix
        
        r_tw, r_dt = regularization_list[it](n_tw, theta_matrix)
        theta_indices = theta_matrix > 0
        r_dt[theta_indices] /= theta_matrix[theta_indices]
        
        g_dt = A.dot(phi_matrix_tr) + r_dt
        tmp = g_dt.T * B / word_norm
        r_tw += (tmp - np.einsum('ij,ji->i', phi_rev_matrix, tmp)) * phi_matrix
        
        n_tw += r_tw
        n_tw[n_tw < 0] = 0
        phi_matrix = n_tw / np.sum(n_tw, axis=1)[:, np.newaxis]
        phi_matrix[np.isnan(phi_matrix)] = 0.

        if iteration_callback is not None:
            iteration_callback(it, phi_matrix, theta_matrix)
    
    print 'Iters time', time.time() - start_time    
    return phi_matrix, theta_matrix

# Gradient Descent

##### Мы можем найти градиент оптимизируемой функции и сделать шаг вдоль него. Сделано для сравнения.

In [11]:
def gradient_optimization(
    n_dw_matrix, 
    phi_matrix,
    theta_matrix,
    regularization_gradient_list,
    iters_count=100,
    loss_function=LogFunction(),
    iteration_callback=None,
    learning_rate=1.
):
    D, W = n_dw_matrix.shape
    T = phi_matrix.shape[0]
    phi_matrix = np.copy(phi_matrix)
    theta_matrix = np.copy(theta_matrix)
    docptr = []
    indptr = n_dw_matrix.indptr
    for doc_num in xrange(D):
        docptr.extend([doc_num] * (indptr[doc_num + 1] - indptr[doc_num]))
    docptr = np.array(docptr)
    wordptr = n_dw_matrix.indices
    
    start_time = time.time()
    for it in xrange(iters_count):
        phi_matrix_tr = np.transpose(phi_matrix)
        # следующая строчка это 60% времени работы алгоритма
        s_data = loss_function.calc_der(inner1d(theta_matrix[docptr, :], phi_matrix_tr[wordptr, :]))
        # следующая часть это 25% времени работы алгоритма
        A = scipy.sparse.csr_matrix(
            (
                n_dw_matrix.data * s_data, 
                n_dw_matrix.indices, 
                n_dw_matrix.indptr
            ), 
            shape=n_dw_matrix.shape
        ).tocsc()
        # Остальное это 15% времени
        g_tw = theta_matrix.T * A
        g_dt = A.dot(phi_matrix_tr)
        
        r_tw, r_dt = regularization_gradient_list[it](phi_matrix, theta_matrix)
        g_tw += r_tw
        g_dt += r_dt
        
        g_tw -= np.sum(g_tw * phi_matrix, axis=1)[:, np.newaxis]
        g_dt -= np.sum(g_dt * theta_matrix, axis=1)[:, np.newaxis]
        
        phi_matrix += g_tw * learning_rate
        theta_matrix += g_dt * learning_rate
        
        phi_matrix[phi_matrix < 0] = 0
        theta_matrix[theta_matrix < 0] = 0
        
        phi_matrix /= np.sum(phi_matrix, axis=1)[:, np.newaxis]
        theta_matrix /= np.sum(theta_matrix, axis=1)[:, np.newaxis]
        
        if iteration_callback is not None:
            iteration_callback(it, phi_matrix, theta_matrix)
    
    print 'Iters time', time.time() - start_time  
    return phi_matrix, theta_matrix

# Оценка качества классификации

In [12]:
def svm_score(theta, targets):
    C_2d_range = [1e-1, 1e0, 1e1, 1e2, 1e3, 1e4]
    gamma_2d_range = [1e-3, 1e-2, 1e-1, 1, 1e1]
    best_C, best_gamma, best_val = None, None, 0.
    best_cv_algo_score_on_test = 0.
    X_train, X_test, y_train, y_test = train_test_split(theta, targets, test_size=0.30, stratify=targets, random_state=42)
    for C in C_2d_range:
        for gamma in gamma_2d_range:
            val = np.mean(cross_val_score(SVC(C=C, gamma=gamma), X_train, y_train, scoring='accuracy', cv=4))
            algo = SVC(C=C, gamma=gamma).fit(X_train, y_train)
            test_score = accuracy_score(y_test, algo.predict(X_test))
            print 'SVM(C={}, gamma={}) cv-score: {}  test-score: {}'.format(
                C,
                gamma,
                round(val, 3),
                round(test_score, 3)
            )
            if val > best_val:
                best_val = val
                best_C = C
                best_gamma = gamma
                best_cv_algo_score_on_test = test_score
    print '\n\n\nBest cv params: C={}, gamma={}\nCV score: {}\nTest score:{}'.format(
        best_C,
        best_gamma,
        round(best_val, 3),
        round(best_cv_algo_score_on_test, 3)
    )
    return best_C, best_gamma, best_val, best_cv_algo_score_on_test

In [13]:
def artm_calc_topic_correlation(phi):
    T, W = phi.shape
    return (np.sum(np.sum(phi, axis=0) ** 2) - np.sum(phi ** 2)) / (T * (T - 1))

In [14]:
def artm_calc_perplexity_factory(n_dw_matrix):
    helper = create_calculate_likelihood_like_function(
        loss_function=LogFunction(),
        n_dw_matrix=n_dw_matrix
    )
    total_words_number = n_dw_matrix.sum()
    return lambda phi, theta: np.exp(- helper(phi, theta) / total_words_number)     

In [41]:
def artm_calc_pmi_top_factory(doc_occurences, doc_cooccurences, documents_number, top_size):
    def fun(phi):
        T, W = phi.shape
        pmi = 0.
        for t in xrange(T):
            top = heapq.nlargest(top_size, xrange(W), key=lambda w: phi[t, w])
            for w1 in top:
                for w2 in top:
                    if w1 != w2:
                        pmi += np.log(documents_number * (doc_cooccurences[(w1, w2)] + 1e-4) * 1. / doc_occurences[w1] / doc_occurences[w2])
        return pmi / (T * top_size * (top_size - 1))
    return fun

# Примеры запусков с разделением на train и test по словам

In [40]:
dataset = fetch_20newsgroups(
    subset='all',
    categories=['sci.electronics', 'sci.med', 'sci.space'],
    remove=('headers', 'footers', 'quotes')
)

In [17]:
%%time
train_n_dw_matrix, test_n_dw_matrix, ttt_token_2_num, ttt_num_2_token, ttt_doc_targets, ttt_doc_occurences, ttt_doc_cooccurences = prepare_dataset(dataset, calc_cooccurences=True, train_test_split=0.8)

Processed:  0 documents from 2961
Processed:  500 documents from 2961
Processed:  1000 documents from 2961
Processed:  1500 documents from 2961
Processed:  2000 documents from 2961
Processed:  2500 documents from 2961
Nonzero values: 130804
CPU times: user 3min 19s, sys: 2.2 s, total: 3min 22s
Wall time: 3min 22s


## PLSA: EM

In [60]:
D, W = train_n_dw_matrix.shape
T = 10

np.random.seed(64)

phi_matrix = np.random.uniform(size=(T, W)).astype(np.float64)
phi_matrix /= np.sum(phi_matrix, axis=1)[:, np.newaxis]

theta_matrix = np.ones(shape=(D, T)).astype(np.float64)
theta_matrix /= np.sum(theta_matrix, axis=1)[:, np.newaxis]

regularization_list = np.zeros(200, dtype=object)
regularization_list[:] = create_reg_lda(0., 0.)

_train_perplexity = artm_calc_perplexity_factory(train_n_dw_matrix) 
_test_perplexity = artm_calc_perplexity_factory(test_n_dw_matrix)
_top5_pmi = artm_calc_pmi_top_factory(ttt_doc_occurences, ttt_doc_cooccurences, train_n_dw_matrix.shape[0], 5)
_top10_pmi = artm_calc_pmi_top_factory(ttt_doc_occurences, ttt_doc_cooccurences, train_n_dw_matrix.shape[0], 10)
_top20_pmi = artm_calc_pmi_top_factory(ttt_doc_occurences, ttt_doc_cooccurences, train_n_dw_matrix.shape[0], 20)

def callback(it, phi, theta):
    if it % 10 == 0:
        print 'Iteration', it
        print '\tTrain perplexity:   {}'.format(_train_perplexity(phi, theta))
        print '\tTest perplexity:     {}'.format(_test_perplexity(phi, theta))
        print '\tTopics correlation  {}'.format(artm_calc_topic_correlation(phi))
        print '\tPhi sparsity        {}'.format(1. * np.sum(phi < 1e-20) / np.sum(phi >= 0))
        print '\tTheta sparsity      {}'.format(1. * np.sum(theta < 0.01) / np.sum(theta >= 0))
        print '\tTop5 PMI            {}'.format(_top5_pmi(phi))
        print '\tTop10 PMI           {}'.format(_top10_pmi(phi))
        print '\tTop20 PMI           {}'.format(_top20_pmi(phi))


phi, theta = em_optimization(
    n_dw_matrix=train_n_dw_matrix, 
    phi_matrix=phi_matrix,
    theta_matrix=theta_matrix,
    regularization_list=regularization_list,
    iters_count=101,
    loss_function=LogFunction(),
    iteration_callback=callback
)

Iteration 0
	Train perplexity:   3189.46334242
	Test perplexity:     3531.73423591
	Topics correlation  0.000725998723954
	Phi sparsity        0.0014551152899
	Theta sparsity      0.000216684723727
	Top5 PMI            0.423826388526
	Top10 PMI           0.468634502156
	Top20 PMI           0.509231453518
Iteration 10
	Train perplexity:   2248.79526472
	Test perplexity:     3076.36649028
	Topics correlation  0.000662166455725
	Phi sparsity        0.00183568390419
	Theta sparsity      0.322246298303
	Top5 PMI            0.701193486577
	Top10 PMI           0.581648355023
	Top20 PMI           0.593460217964
Iteration 20
	Train perplexity:   1799.93196596
	Test perplexity:     2958.83947081
	Topics correlation  0.000620877413692
	Phi sparsity        0.155238415044
	Theta sparsity      0.584832069339
	Top5 PMI            0.934272740933
	Top10 PMI           0.920020291533
	Top20 PMI           0.805215334693
Iteration 30
	Train perplexity:   1691.70832655
	Test perplexity:     3242.32525268
	T

In [61]:
for t in xrange(T):
    print t
    top = heapq.nlargest(20, xrange(W), key=lambda w: phi[t, w])
    print '\n'.join('\t' + x[:-3] for x in map(ttt_num_2_token.get, top))

0
	space
	available
	theory
	system
	also
	information
	image
	keyboard
	edu
	book
	nasa
	astronomy
	time
	pub
	star
	datum
	archive
	universe
	ftp
	list
1
	used
	get
	know
	battery
	power
	thank
	circuit
	radio
	chip
	use
	good
	anyone
	edu
	output
	input
	make
	detector
	voltage
	time
	need
2
	patient
	treatment
	study
	vitamin
	food
	disease
	also
	effect
	drug
	diet
	cause
	get
	know
	infection
	medical
	person
	used
	cancer
	body
	good
3
	health
	hiv
	medical
	disease
	number
	april
	research
	page
	center
	year
	cancer
	aids
	information
	national
	university
	patient
	newsletter
	child
	volume
	person
4
	get
	person
	think
	go
	say
	know
	see
	thing
	take
	time
	make
	seem
	year
	problem
	day
	much
	even
	something
	doctor
	try
5
	mission
	earth
	orbit
	space
	moon
	solar
	planet
	lunar
	probe
	system
	venus
	mar
	surface
	spacecraft
	first
	atmosphere
	year
	shuttle
	make
	satellite
6
	edu
	com
	post
	know
	phone
	group
	anyone
	internet
	thank
	email
	get
	net
	board
	mail
	ne

## PLSA: Thetaless EM

In [58]:
D, W = train_n_dw_matrix.shape
T = 10

np.random.seed(64)

phi_matrix = np.random.uniform(size=(T, W)).astype(np.float64)
phi_matrix /= np.sum(phi_matrix, axis=1)[:, np.newaxis]

theta_matrix = np.ones(shape=(D, T)).astype(np.float64)
theta_matrix /= np.sum(theta_matrix, axis=1)[:, np.newaxis]

regularization_list = np.zeros(200, dtype=object)
regularization_list[:] = create_reg_lda(0., 0.)

_train_perplexity = artm_calc_perplexity_factory(train_n_dw_matrix) 
_test_perplexity = artm_calc_perplexity_factory(test_n_dw_matrix)
_top5_pmi = artm_calc_pmi_top_factory(ttt_doc_occurences, ttt_doc_cooccurences, train_n_dw_matrix.shape[0], 5)
_top10_pmi = artm_calc_pmi_top_factory(ttt_doc_occurences, ttt_doc_cooccurences, train_n_dw_matrix.shape[0], 10)
_top20_pmi = artm_calc_pmi_top_factory(ttt_doc_occurences, ttt_doc_cooccurences, train_n_dw_matrix.shape[0], 20)

def callback(it, phi, theta):
    if it % 10 == 0:
        print 'Iteration', it
        print '\tTrain perplexity:   {}'.format(_train_perplexity(phi, theta))
        print '\tTest perplexity:     {}'.format(_test_perplexity(phi, theta))
        print '\tTopics correlation  {}'.format(artm_calc_topic_correlation(phi))
        print '\tPhi sparsity        {}'.format(1. * np.sum(phi < 1e-20) / np.sum(phi >= 0))
        print '\tTheta sparsity      {}'.format(1. * np.sum(theta < 0.01) / np.sum(theta >= 0))
        print '\tTop5 PMI            {}'.format(_top5_pmi(phi))
        print '\tTop10 PMI           {}'.format(_top10_pmi(phi))
        print '\tTop20 PMI           {}'.format(_top20_pmi(phi))


phi, theta = artm_thetaless_em_optimization(
    n_dw_matrix=train_n_dw_matrix, 
    phi_matrix=phi_matrix,
    regularization_list=regularization_list,
    iters_count=101,
    iteration_callback=callback
)

Iteration 0
	Train perplexity:   3185.57389908
	Test perplexity:     3529.86337686
	Topics correlation  0.00072332950865
	Phi sparsity        0.0014551152899
	Theta sparsity      0.000216684723727
	Top5 PMI            0.42418486873
	Top10 PMI           0.471481807837
	Top20 PMI           0.50969968015
Iteration 10
	Train perplexity:   2295.8825486
	Test perplexity:     2803.53781419
	Topics correlation  0.000555588587629
	Phi sparsity        0.282012536378
	Theta sparsity      0.0150957024196
	Top5 PMI            0.821836003313
	Top10 PMI           0.824893165098
	Top20 PMI           0.871302561325
Iteration 20
	Train perplexity:   1986.3987337
	Test perplexity:     2581.85168597
	Topics correlation  0.00031856497952
	Phi sparsity        0.400526080143
	Theta sparsity      0.0900325027086
	Top5 PMI            0.965319406083
	Top10 PMI           1.11859956138
	Top20 PMI           1.07747160688
Iteration 30
	Train perplexity:   1913.24748471
	Test perplexity:     2547.76185602
	Topics co

In [59]:
for t in xrange(T):
    print t
    top = heapq.nlargest(20, xrange(W), key=lambda w: phi[t, w])
    print '\n'.join('\t' + x[:-3] for x in map(ttt_num_2_token.get, top))

0
	available
	book
	datum
	image
	theory
	reference
	general
	field
	system
	base
	astronomy
	keyboard
	star
	ray
	experiment
	word
	radar
	element
	pc
	position
1
	anyone
	thank
	look
	line
	radio
	use
	chip
	buy
	run
	battery
	output
	lead
	soon
	frequency
	like
	input
	appreciate
	signal
	detector
	voltage
2
	patient
	cause
	effect
	disease
	doctor
	food
	study
	treatment
	drug
	infection
	vitamin
	body
	diet
	medical
	month
	medicine
	clinical
	cell
	pain
	eat
3
	information
	number
	health
	research
	center
	april
	medical
	hiv
	university
	national
	include
	page
	year
	child
	state
	aids
	volume
	cancer
	issue
	report
4
	get
	know
	time
	make
	person
	go
	see
	say
	think
	take
	good
	problem
	thing
	find
	want
	well
	many
	way
	even
	give
5
	earth
	mission
	orbit
	moon
	solar
	spacecraft
	planet
	lunar
	probe
	first
	system
	surface
	mar
	degree
	venus
	energy
	sun
	shuttle
	mass
	mile
6
	edu
	com
	mail
	post
	name
	send
	phone
	group
	company
	article
	please
	email
	board
	int

In [135]:
np.sum(np.sum(phi, axis=0) < 1e-10)

13

In [58]:
np.sum(np.sum(theta, axis=1) < 0.9999)

0

# Разделение на train test по документам

In [84]:
dataset_train = fetch_20newsgroups(
    subset='train',
    categories=['sci.electronics', 'sci.med', 'sci.space', 'sci.crypt', 'rec.sport.baseball', 'rec.sport.hockey'],
    remove=('headers', 'footers', 'quotes')
)

In [85]:
%%time
n_dw_matrix_doc_train, token_2_num_doc_train, num_2_token_doc_train, doc_targets_doc_train = prepare_dataset(dataset_train)

Processed:  0 documents from 3570
Processed:  500 documents from 3570
Processed:  1000 documents from 3570
Processed:  1500 documents from 3570
Processed:  2000 documents from 3570
Processed:  2500 documents from 3570
Processed:  3000 documents from 3570
Processed:  3500 documents from 3570
Nonzero values: 199636
CPU times: user 4min 3s, sys: 96 ms, total: 4min 4s
Wall time: 4min 4s


In [86]:
dataset_test = fetch_20newsgroups(
    subset='test',
    categories=['sci.electronics', 'sci.med', 'sci.space', 'sci.crypt', 'rec.sport.baseball', 'rec.sport.hockey'],
    remove=('headers', 'footers', 'quotes')
)

In [87]:
%%time
n_dw_matrix_doc_test, token_2_num_doc_test, num_2_token_doc_test, doc_targets_doc_test = prepare_dataset(dataset_test, token_2_num=token_2_num_doc_train)

Nonzero values: 109917
CPU times: user 1min 8s, sys: 8 ms, total: 1min 8s
Wall time: 1min 8s


In [113]:
D, W = n_dw_matrix_doc_train.shape
T = 5

np.random.seed(47)

phi_matrix = np.random.uniform(size=(T, W)).astype(np.float64)
phi_matrix /= np.sum(phi_matrix, axis=1)[:, np.newaxis]

theta_matrix = np.ones(shape=(D, T)).astype(np.float64)
theta_matrix /= np.sum(theta_matrix, axis=1)[:, np.newaxis]

regularization_list = np.zeros(200, dtype=object)
regularization_list[:] = trivial_regularization

phi, theta = em_optimization(
    n_dw_matrix=n_dw_matrix_doc_train, 
    phi_matrix=phi_matrix,
    theta_matrix=theta_matrix,
    regularization_list=regularization_list,
    iters_count=150,
    loss_function=LogFunction()
)

best_C, best_gamma, _, _ = svm_score(theta, doc_targets_doc_train)
algo = SVC(C=best_C, gamma=best_gamma).fit(theta, doc_targets_doc_train)

Iters time 4.03662395477
SVM(C=0.1, gamma=0.001) cv-score: 0.208  test-score: 0.169
SVM(C=0.1, gamma=0.01) cv-score: 0.208  test-score: 0.169
SVM(C=0.1, gamma=0.1) cv-score: 0.643  test-score: 0.65
SVM(C=0.1, gamma=1) cv-score: 0.676  test-score: 0.685
SVM(C=0.1, gamma=10.0) cv-score: 0.69  test-score: 0.7
SVM(C=1.0, gamma=0.001) cv-score: 0.208  test-score: 0.169
SVM(C=1.0, gamma=0.01) cv-score: 0.643  test-score: 0.652
SVM(C=1.0, gamma=0.1) cv-score: 0.677  test-score: 0.683
SVM(C=1.0, gamma=1) cv-score: 0.688  test-score: 0.7
SVM(C=1.0, gamma=10.0) cv-score: 0.695  test-score: 0.704
SVM(C=10.0, gamma=0.001) cv-score: 0.643  test-score: 0.653
SVM(C=10.0, gamma=0.01) cv-score: 0.678  test-score: 0.682
SVM(C=10.0, gamma=0.1) cv-score: 0.687  test-score: 0.7
SVM(C=10.0, gamma=1) cv-score: 0.688  test-score: 0.709
SVM(C=10.0, gamma=10.0) cv-score: 0.685  test-score: 0.697
SVM(C=100.0, gamma=0.001) cv-score: 0.677  test-score: 0.683
SVM(C=100.0, gamma=0.01) cv-score: 0.689  test-score: 0.

In [114]:
D, W = n_dw_matrix_doc_test.shape
theta_matrix = np.ones(shape=(D, T)).astype(np.float64)
theta_matrix /= np.sum(theta_matrix, axis=1)[:, np.newaxis]

print 'PLSA not const phi' 
for iters in xrange(1, 6):
    _, theta_test = em_optimization(
        n_dw_matrix=n_dw_matrix_doc_test, 
        phi_matrix=phi,
        theta_matrix=theta_matrix,
        regularization_list=regularization_list,
        iters_count=iters,
        loss_function=LogFunction(),
    )
    print iters, accuracy_score(algo.predict(theta_test), doc_targets_doc_test)

print '\nPLSA const phi' 
for iters in xrange(1, 6):
    _, theta_test = em_optimization(
        n_dw_matrix=n_dw_matrix_doc_test, 
        phi_matrix=phi,
        theta_matrix=theta_matrix,
        regularization_list=regularization_list,
        iters_count=iters,
        loss_function=LogFunction(),
        const_phi=True
    )
    print iters, accuracy_score(algo.predict(theta_test), doc_targets_doc_test)
    
print '\nARTM thetaless'    
for iters in xrange(1, 6):
    _, theta_test = artm_thetaless_em_optimization(
        n_dw_matrix=n_dw_matrix_doc_test, 
        phi_matrix=phi,
        regularization_list=regularization_list,
        iters_count=iters,
    )
    print iters, accuracy_score(algo.predict(theta_test), doc_targets_doc_test)

PLSA not const phi
Iters time 0.0184760093689
1 0.498251748252
Iters time 0.0313498973846
2 0.613199300699
Iters time 0.0481491088867
3 0.644667832168
Iters time 0.0741291046143
4 0.666520979021
Iters time 0.0801839828491
5 0.683129370629

PLSA const phi
Iters time 0.014631986618
1 0.498251748252
Iters time 0.0475838184357
2 0.626311188811
Iters time 0.0461721420288
3 0.666083916084
Iters time 0.0600559711456
4 0.690996503497
Iters time 0.0745630264282
5 0.692307692308

ARTM thetaless
Iters time 0.0183432102203
1 0.498251748252
Iters time 0.0338151454926
2 0.54458041958
Iters time 0.0561671257019
3 0.564248251748
Iters time 0.080549955368
4 0.57736013986
Iters time 0.104263067245
5 0.583916083916


In [115]:
D, W = n_dw_matrix_doc_train.shape
T = 5

np.random.seed(47)

phi_matrix = np.random.uniform(size=(T, W)).astype(np.float64)
phi_matrix /= np.sum(phi_matrix, axis=1)[:, np.newaxis]

theta_matrix = np.ones(shape=(D, T)).astype(np.float64)
theta_matrix /= np.sum(theta_matrix, axis=1)[:, np.newaxis]

regularization_list = np.zeros(200, dtype=object)
regularization_list[:] = trivial_regularization

phi, theta = artm_thetaless_em_optimization(
    n_dw_matrix=n_dw_matrix_doc_train, 
    phi_matrix=phi_matrix,
    regularization_list=regularization_list,
    iters_count=150
)

best_C, best_gamma, _, _ = svm_score(theta, doc_targets_doc_train)
algo = SVC(C=best_C, gamma=best_gamma).fit(theta, doc_targets_doc_train)

Iters time 4.7924489975
SVM(C=0.1, gamma=0.001) cv-score: 0.206  test-score: 0.169
SVM(C=0.1, gamma=0.01) cv-score: 0.206  test-score: 0.169
SVM(C=0.1, gamma=0.1) cv-score: 0.543  test-score: 0.588
SVM(C=0.1, gamma=1) cv-score: 0.63  test-score: 0.633
SVM(C=0.1, gamma=10.0) cv-score: 0.638  test-score: 0.643
SVM(C=1.0, gamma=0.001) cv-score: 0.206  test-score: 0.169
SVM(C=1.0, gamma=0.01) cv-score: 0.541  test-score: 0.592
SVM(C=1.0, gamma=0.1) cv-score: 0.634  test-score: 0.633
SVM(C=1.0, gamma=1) cv-score: 0.644  test-score: 0.646
SVM(C=1.0, gamma=10.0) cv-score: 0.639  test-score: 0.652
SVM(C=10.0, gamma=0.001) cv-score: 0.541  test-score: 0.595
SVM(C=10.0, gamma=0.01) cv-score: 0.634  test-score: 0.633
SVM(C=10.0, gamma=0.1) cv-score: 0.644  test-score: 0.646
SVM(C=10.0, gamma=1) cv-score: 0.635  test-score: 0.658
SVM(C=10.0, gamma=10.0) cv-score: 0.643  test-score: 0.653
SVM(C=100.0, gamma=0.001) cv-score: 0.635  test-score: 0.633
SVM(C=100.0, gamma=0.01) cv-score: 0.642  test-sco

In [116]:
D, W = n_dw_matrix_doc_test.shape
theta_matrix = np.ones(shape=(D, T)).astype(np.float64)
theta_matrix /= np.sum(theta_matrix, axis=1)[:, np.newaxis]

print 'PLSA not const phi' 
for iters in xrange(1, 6):
    _, theta_test = em_optimization(
        n_dw_matrix=n_dw_matrix_doc_test, 
        phi_matrix=phi,
        theta_matrix=theta_matrix,
        regularization_list=regularization_list,
        iters_count=iters,
        loss_function=LogFunction(),
    )
    print iters, accuracy_score(algo.predict(theta_test), doc_targets_doc_test)

print '\nPLSA const phi' 
for iters in xrange(1, 6):
    _, theta_test = em_optimization(
        n_dw_matrix=n_dw_matrix_doc_test, 
        phi_matrix=phi,
        theta_matrix=theta_matrix,
        regularization_list=regularization_list,
        iters_count=iters,
        loss_function=LogFunction(),
        const_phi=True
    )
    print iters, accuracy_score(algo.predict(theta_test), doc_targets_doc_test)
    
print '\nARTM thetaless'    
for iters in xrange(1, 6):
    _, theta_test = artm_thetaless_em_optimization(
        n_dw_matrix=n_dw_matrix_doc_test, 
        phi_matrix=phi,
        regularization_list=regularization_list,
        iters_count=iters,
    )
    print iters, accuracy_score(algo.predict(theta_test), doc_targets_doc_test)

PLSA not const phi
Iters time 0.0153000354767
1 0.621940559441
Iters time 0.034432888031
2 0.622814685315
Iters time 0.0490560531616
3 0.615384615385
Iters time 0.067636013031
4 0.61756993007
Iters time 0.0804588794708
5 0.620192307692

PLSA const phi
Iters time 0.0172531604767
1 0.621940559441
Iters time 0.0339169502258
2 0.622377622378
Iters time 0.0523760318756
3 0.620192307692
Iters time 0.0636301040649
4 0.616258741259
Iters time 0.0816099643707
5 0.615384615385

ARTM thetaless
Iters time 0.0220048427582
1 0.621940559441
Iters time 0.0384120941162
2 0.623688811189
Iters time 0.0598139762878
3 0.621503496503
Iters time 0.0781240463257
4 0.618881118881
Iters time 0.0938749313354
5 0.620629370629
